# Базы данных

____
Для работы с базами данных одним из популярных инструментов является `DBeaver` (Database Management Tool).   
Он поддерживает разные базы.   
Реляционные (SQL) - `MySQL, PostgreSQL, Microsoft SQL Server, Oracle, SQLite`   
Нереляционные (NoSQL) - `ClickHouse, MongoDB, Redis, Cassandra`

Реляционную базу данных можно представить в виде таблиц. В них набор данных. Есть `столбцы` и `строки`.   
! В каждом столбце могут храниться данные строго одного типа.   
Основные это - `int`, `float`, `str` и булев тип `True` и `False`.   
Кроме этих базовых типов есть еще специальные - `JSON` и `XML`.       

**SQLite** самая простенькая и компактная база данных, но она поддерживает всего 5 типов данных: `INTEGER, REAL, TEXT, BLOB, NULL`   
**Реляционная база данных** может содержать множество таблиц и они все могут быть связаным между собой.   
Как пример можно привести две таблицы:
- Таблица с режиссерами где есть **id** и **Имя режиссера**.   
- Таблица с фильмами, которая ссылается на таблицу с режжисерами по `foreign key (FK)`. То есть в ней будет колонка в конце таблицы, где будут написаны **id** режиссеров.   

`Нереляционные` базы данных не требуют четкой структуры. Так как информация может быть разнородной, существуют `NoSQL - (Нереляционные БД)`.   
- Например **Redis** это база типа ключ-значение. Обычно такая база используется для кэширования. Преимущества таких баз в скорости. Они дают очень быстрый доступ к информации.      
- Или **MongoDB** он больше для документов. То есть нахождение информации из разных доков.   

`Пример SQL запроса`. Тут все логично и понятно. Комментарии ставят с помощью /* и */ для закрытия.   

In [ ]:
SELECT title FROM movies WHERE product_type = 'Сериал' AND release_year < 1991;

Ключевые слова для запросов: **SELECT, COUNT, ASC, KEY, INSERT, UPDATE, DELETE** и др.   
Ключевые операторы: **<, >, =, LIKE, IN, AND** и др.

У нас в DBeaver есть базовая тренировочная база, к примеру. `DBeaver Sample Database`. В ней есть папка с таблицами. Чтобы писать запрос надо нажать **SQL** -> Открыть SQL скрипт и в вышедшем окне писать запросы. Редактировать таблицы можно и вручную, а можно и вот так, через запросы. Это касается и удаления и прочего. После каждого действия `F5` чтобы обновить вид.   
`Если` мы пишем много запросов, просто надо выполнить скрипт целиком чтобы весь код выполнился.   

In [ ]:
CREATE TABLE video_products (title TEXT, release_year INTEGER);
INSERT INTO video_products VALUES('Dictator', 2010);
INSERT INTO video_products VALUES('Rambo', 2011);

Ниже примеры работы с таблицей. Что мы из нее хотим получить и по каким параметрам.   

In [ ]:
SELECT * FROM video_products;
SELECT title FROM video_products WHERE release_year = 2010;

Стоит отметить что не обязательно всегда открывать БД в DBeaver. Наши таблицы можно смотреть и в редакторе кода. Через расширения и драйверы можно подключаться к любой БД и работать с ней.   
- Для начала база должна быть на компьютере локально (Для SQLite это**файлы .sqlite или .db**) или на сервере. Для `PostgreSQL` — сервер на локальном ПК или удалённый.   

`import sqlite3` — это встроенный модуль Python для работы с SQLite.   
Для PostgreSQL в Python используется другой драйвер, чаще всего это `psycopg2` или его современный вариант `psycopg`    
Надо будет просто установить через **pip install psycopg2**   

In [ ]:
import psycopg2   # Это для PostgreSQL
import sqlite3    # Это для SQLite

con = psycopg2.connect(
    dbname="mydatabase",
    user="myuser",
    password="mypassword",
    host="localhost",
    port="5432"
)

# con = sqlite3.connect('db.sqlite')  <- это для SQLite. Если с таким именем БД нет, он будет создан автоматически.

# Создание курсора (обязательно для работы с БД)
cur = con.cursor()

# Пример запроса
# Для читаемости запрос обрамлён в тройные кавычки и разбит построчно.
query = '''
CREATE TABLE IF NOT EXISTS directors(
    id INTEGER PRIMARY KEY,
    full_name TEXT,
    birth_year INTEGER
);

CREATE TABLE IF NOT EXISTS video_products(
    id INTEGER PRIMARY KEY,
    title TEXT,
    product_type TEXT,
    release_year INTEGER
);
'''

cur.executescript(query)  # Сколько у нас SQL запросов, то есть query, столько раз и нужно execute. cur.execute(query) Если запрос один.
con.close()

Также полезно знать что БД можно создавать в памяти компьютера. После закрытия соединения БД удаляется. Это полезно для отладки и тестирования. Такие БД называются `Резидентные`.   

In [ ]:
con = sqlite3.connect(':memory:')

С заполнением таблицы нужно быть аккуратнее. Нельзя в метод execute передавать обычную f строку, так как такое могут взломать. Для этого в метод execute значение полей передовать можно в виде кортежа. В формате **ниже** с вопросительными знаками никто взломать не сможет.   

In [ ]:
directors = [
    (1, 'Тайка Вайтити', 1975),
    (2, 'Квентин Тарантино', 1963),
    (3, 'Дамьен Шазелл', 1985),
]
video_products = [
    (1, 'УБить Билла', 'Фильм', 2003),
    (2, 'Ла-Ла-Лэнд', 'Фильм', 2016),
    (3, 'Кролик Джоджо', 'Фильм', 2019),
    (4, 'Бесславные ублюдки', 'Фильм', 2009)
]

cur.executemany('INSERT INTO directors VALUES(?, ?, ?);', directors)
cur.executemany('INSERT INTO video_products VALUES(?, ?, ?, ?);', video_products)

con.commit()   # Надо вызвать этот метод, чтобы изменения сохранились.
con.close()

Далее запрос к БД на получения нужных нам данных. Сначала идёт **SELECT** потом **FROM** потом только **WHERE**.   

In [ ]:
import sqlite3

con = sqlite3.connect('db_video.sqlite')
cur = con.cursor()

results = cur.execute('''
    SELECT title,
           release_year
    FROM video_products
    WHERE release_year > 2010;
''')
# WHERE release_year BETWEEN 2010 AND 2016;
# WHERE release_year != 2019;
# WHERE product_type IN ('Сериал', 'Фильм');
# WHERE product_type LIKE 'Фил%';
# WHERE (product_type = 'Фильм' OR product_type = 'Сериал') AND release_year = 2019;

for result in results:
    print(result)

# con.commit() не нужен когда мы хотим получить данные из таблицы
con.close()

Есть специальное значение `NULL`. Оно ничему не равно. Это просто пустота. Если вы даже захотите вызвать столбцы где есть NULL - у вас ничего не выйдет. NULL используют например если у нас столбец с номерами телефона, но есть люди чьи номера пока не известны.   
Еще можно например получить уникальные значения из столбца с помощью `SELECT DISTINCT`. Так нам вернет `('Фильм')`

Во время запроса, данные могут выводиться вразнобой. Для этого нужна сортировка с помощью `ORDER BY` который пишется после `WHERE`.   
По умолчанию сортировка идет по возрастанию. Если надо чтобы было по убыванию - используем ключевое слово `ASC`. `DESC` - по убыванию.   
Есть еще такие параметры как `LIMIT` - сколько записей надо вернуть и `OFFSET` - сколько записей нужно пропустить.   
То есть если мы пишем `LIMIT 2 OFFSET 3;`, то нам вернутся две записи, но первые 3 сверху будут пропущены.   

In [ ]:
ORDER BY release_year DESC
LIMIT 2 OFFSET 2;

`Агрегирующие` функции.   
Они нужны для более глубокой выборки. Например нам нужно собрать кассовые сборы по фильмам за 2019 год и такого рода вещи.   
Можно еще искать `MIN` и `MAX` значения.
Можно еще вывести среднее `AVG` или сумму `SUM`.   

In [ ]:
SELECT COUNT(gross)
FROM video_products;

SELECT MIN(gross)
FROM video_products;

SELECT AVG(gross)
FROM video_products
WHERE release_year > 2010;

SELECT SUM(gross)
FROM video_products
WHERE release_year > 2019;

Или например нам надо посчитать сколько всего у нас именно фильмов.

In [ ]:
SELECT COUNT(*)
FROM video_products
WHERE product_type='Фильм';

Также можно группировать записи, у которых совпадение по значениям с помощью `GROUP BY`.   
Например так можно узнать сколько у нас фильмов, сериалов и мультфильмов.   

In [ ]:
SELECT product_type,
       COUNT(*)
FROM video_products
GROUP BY product_type;

Или вот еще
SELECT product_type,         Выбрать тип
       SUM(gross)            Просуммировать кассовые сборы
FROM video_products
WHERE release_year > 2017    Где дата выхода больше 2017 года
GROUP BY product_type;       Сгруппированно по типу

Существует еще фильтрация `HAVING`. Она тоже для филтрации групп.   

In [ ]:
SELECT product_type,
       SUM(gross)
FROM video_products
GROUP BY product_type
HAVING SUM(gross) IS NOT NULL;   Исключим все группы, чья сумма кассовых сборов равна NULL

Правильный порядок SQL запроса:   
- `SELECT` - обязательно. К нему могут идти необязательные `COUNT`, `MIN`, `MAX`, `AVG`, `SUM` и ключевое слово `DISTINCT`.   
- `FROM` таблица - обязательно.
- `WHERE` условие или фильтрация таблицы - необязательно.
- `GROUP BY` столбец или несколько по которым группируется - необязательно.
- `HAVING` условие и фильтрация на уровне сгруппированных записей - необязательно.
- `ORDER BY` столбец или столбцы по которым сортируется вывод - необязательно. К нему `DESC` - по убыванию, если надо.
- `LIMIT` сколько записей показать - необязательно.
- `OFFSET` сколько записей пропустить - необязательно

`Связи между таблицами`.   
Существует три вида связей:   
- Один к одному 1:1_______**(Например Режиссер - Паспорт id)**
- Многие к одному M:1____**(Например Режиссер - место рождения) Может быть одно место для многих.**
- Многие к многим N:M____**(Например Режиссеры - фильмы) Может быть много режиссеров с разными фильмами и у одного фильма могут быть много режиссеров.**
   
Для того чтобы построить эти связи в каждой таблице должны быть поля и столбцы с **primary key**(должен быть уникальным). А в таблицах, которые ссылаются на другие - **foreign key** (может повторяться).   
На уровне базы данных `нельзя` установить прямую связь `N:M` между двумя таблицами. Для такой связи необходима `промежуточная таблица`.   
Посредством этой таблицы можно связать один фильм с несколькими режиссёрами и одного режиссёра — с несколькими фильмами. Обратившись к этой таблице, можно получить информацию о связях.  

`1:1 Один к одному` и `M:1 Многие к одному` на SQLite.   
Сначала создаем три таблицы: Таблица с названиями фильмов на русском языке, таблица с названиями фильмов на англ и таблица с жанрами.

In [ ]:
import sqlite3

con = sqlite3.connect('db.sqlite')
cur = con.cursor()

cur.executescript('''
CREATE TABLE IF NOT EXISTS original_titles(
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS product_types(
    id INTEGER PRIMARY KEY,
    type_name TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS video_products(
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    original_title_id INTEGER NOT NULL UNIQUE,
    product_type_id INTEGER NOT NULL,

    FOREIGN KEY(original_title_id) REFERENCES original_titles(id),
    FOREIGN KEY(product_type_id) REFERENCES product_types(id)
);
''')

con.close()


Далее заполняем сами таблицы:   

In [ ]:
import sqlite3

con = sqlite3.connect('db.sqlite')
cur = con.cursor()

original_titles = [
    (1, 'Game of Thrones'),
    (2, 'Inglourious Basterds'),
    (3, 'Zootopia'),
    (4, 'The Last Samurai'),
    (5, 'Troy'),
    (6, 'The Hateful Eight'),
    (7, 'Gladiator')
]

video_products = [
    (1, 'Бесславные ублюдки', 2, 2),
    (2, 'Омерзительная восьмерка', 6, 2),
    (3, 'Игра престолов', 1, 3),
    (4, 'Зверополис', 3, 1),
    (5, 'Последний самурай', 4, 2),
    (6, 'Гладиатор', 7, 2),
    (7, 'Троя', 5, 2)
]

product_types = [
    (1, 'Мультфильм'),
    (2, 'Фильм'),
    (3, 'Сериал'),
]

cur.executemany('INSERT INTO original_titles VALUES (?, ?);', original_titles)
cur.executemany('INSERT INTO product_types VALUES (?, ?);', product_types)
cur.executemany('INSERT INTO video_products VALUES (?, ?, ?, ?);', video_products)

con.commit()
con.close()


Теперь можно вернуть нужные нам поля.   

In [ ]:
SELECT video_products.title AS translation,       <- Можно давать краткие названия
       original_titles.title AS original
FROM video_products,
     original_titles
WHERE
    video_products.original_title_id = original_titles.id
  AND
    original LIKE 'T%'

Или такой запрос
SELECT video_products.name,
       product_types.name
FROM video_products,
     product_types
WHERE video_products.type_id = product_types.id AND product_types.name = 'Фильм';

Принято отделять фильтрацию от условий соединения таблиц с помощью инструкции `JOIN`.   
`INNER` = только пересечение   
`LEFT`  = левое + пересечение   
`RIGHT` = правое + пересечение   
`FULL`  = всё   
`CROSS` = всё × всё   

У нас например есть 3 таблицы:   
**employees** со столбцами `id, name`   
**departments** со столбцами `id, dept_name`   
**employee_department** со столбцами `employee_id, department_id`.   Тут первое ссылается на первую таблицу employees и второе на departments.   

1) `INNER JOIN` выведет нам только тех работников, у кого есть отдел. e, d тут короткое название чтобы не писать длинные строки.   
Вывод будет например name Artur dept_name HR 

In [ ]:
SELECT e.name, d.dept_name                                     Выведи name и dept_name
FROM employees e                                               Из таблицы employees
JOIN employee_department ed ON e.id = ed.employee_id           Соединение с таблицей employee_department, где в таблице employees id = id в таблице employee_department
JOIN departments d ON d.id = ed.department_id;                 Соединение с таблицей departmnets, где в таблице departments id = id в таблице employee_department

2) `LEFT JOIN` выведет всех сотрудников + отдел, если есть, если нет - **NULL**   
Код будет тот же самый что и сверху, только вместо `JOIN` будет `LEFT JOIN`

3) `RIGHT JOIN` выведет все отделы + сотрудника, если он есть, если нет - `NULL`   
Код будет тот же самый что и сверху, только вместо `LEFT JOIN` будет `RIGHT JOIN` 

4) `FULL JOIN` выведет всех сотрудников + все отделы. `NULL` будет там, где у сотрудника нет отдела, или у отдела, у которого нет сотрудника.      
Код будет тот же самый что и сверху, только вместо `RIGHT JOIN` будет `FULL JOIN` 

5) `CROSS JOIN` редко используется. Это просто декартово перемножение. Все сотрудники на все отделы. То есть будет **Atrur** столько раз сколько и отделов.  `NULL` не будет.      
Код будет другим.   

In [ ]:
SELECT e.name, d.dept_name
FROM employees e
CROSS JOIN departments d;

Связь многие к многим - `N:M`   
На том же примере с режиссерами и фильмами.   
Надо будет создать промежуточную таблицу, которая названа по именам таблиц, которые через нее связаны.   

In [ ]:
CREATE TABLE IF NOT EXISTS video_products(
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS directors(
    id INTEGER PRIMARY KEY,
    full_name TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS directors__video_products(              Это наша промежуточная таблица.
    director_id INTEGER NOT NULL,                                  Создаем колонки id режиссера и id фильма/сериала
    video_product_id INTEGER NOT NULL,
    PRIMARY KEY (director_id, video_product_id),                   Первичные ключи во временной таблице
    FOREIGN KEY(director_id) REFERENCES directors(id),             И Внешние ключи которые связывают наши перчивные ключи с id в других таблицах
    FOREIGN KEY(video_product_id) REFERENCES video_products(id)
);

Вообще, в реальных бизнесах и проектах, если БД спроектирована правильно и в ее архитектуре учтены все возможные варианты развития, таблицы можно создать и забыть про них. Но это зачастую миф, так как бизнесы меняются и расширяются. То есть надо будет раширить БД, добавлять новые данные и прочее. Иногда придется удалять таблицы, иногда менять и так далее.   
Для этого используется `ALTER TABLE`

In [ ]:
ALTER TABLE <имя таблицы>                      Переименование таблицы
RENAME TO <новое имя таблицы>;

ALTER TABLE <название таблицы>                 Добавление колонки
ADD COLUMN <имя колонки> <тип колонки>;

ALTER TABLE <название таблицы>                 Переименование колонки
RENAME COLUMN <старое имя колонки>
TO <новое имя колонки>;

ALTER TABLE <название таблицы>                 Удаление колонки
DROP COLUMN <имя колонки>;

DROP TABLE <имя таблицы>;                      Удаление таблицы

**Важно** понимать, что при удалении или переименовании могут возникнуть проблемы и БД перестанет работать.   
То есть происходит нарушение целостности. Важно чтобы не было ведущих в никуда ссылок. Есть три примера нарушения целостности:   
- Аномалия удаления (`deletion anomaly`): если из таблицы удалена строка, на которую ссылается внешний ключ.
- Аномалия вставки (`insertion anomaly`): если добавлена запись с внешним ключом, но этот внешний ключ не соответствует ни одному первичному ключу из связанной таблицы.
- Аномалии обновления (`update anomaly`): при изменении данных в одной строке они могут прийти в противоречие с данными из другой строки: например, изменён PK, на который ссылается FK из другой таблицы.